### 7.10.1 一个参数化的注册装饰器

In [2]:
# registry现在是一个set对象，这样添加和删除函数的速度更快。
registry = set()
# 接受一个可选的关键字参数
def register(active=True):
    # 
    def decorate(func):
        print('running register(active=%s)->decorate(%s)'
             %(active, func))
        if active:
            # 只有active参数的值（从闭包中获取）是True时才注册func。
            registry.add(func)
        else:
            # 如果active不为真，而且func在registry中，那么把它删除。
            registry.discard(func)
        # decorate是装饰器，必须返回一个函数
        return func
    # register是装饰器工厂函数，因此返回decorate
    return decorate

# @register工厂函数必须作为函数调用，并且传入所需的参数。
@register(active=False)
def f1():
    print('running f1（　）')
# 即使不传入参数，register也必须作为函数调用（@register（　）），即要返回真正的装饰器decorate。
@register()
def f2():
    print('running f2（　）')
def f3():
    print('running f3（　）')

running register(active=False)->decorate(<function f1 at 0x0000017668B9CEE0>)
running register(active=True)->decorate(<function f2 at 0x0000017668BD04C0>)


In [3]:
# 导入上述代码

import scripts.registration_param as registration_param

running register(active=False)->decorate(<function f1 at 0x0000017668A599D0>)
running register(active=True)->decorate(<function f2 at 0x0000017668BD0940>)


In [4]:
registration_param.registry

{<function scripts.registration_param.f2()>}

In [6]:
from scripts.registration_param import *

In [7]:
registry

{<function scripts.registration_param.f2()>}

In [8]:
# 将register应用在f3上
register()(f3)

running register(active=True)->decorate(<function f3 at 0x0000017668A59430>)


<function scripts.registration_param.f3()>

In [9]:
registry

{<function scripts.registration_param.f2()>,
 <function scripts.registration_param.f3()>}

In [10]:
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x0000017668BD0940>)


<function scripts.registration_param.f2()>

In [11]:
registry

{<function scripts.registration_param.f3()>}

### 7.10.2 参数化clock装饰器

In [12]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args})-> {result}'
# 参数化的装饰器工厂函数
def clock(fmt=DEFAULT_FMT):
    # 真正的装饰器
    def decorate(func):
        # 被装饰的函数
        def clocked(*_args):
            t0 = time.time()
            # 真正的函数返回的结果
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)
for i in range(3):
    snooze(.123)

[0.13592505s] snooze(0.123)-> None
[0.12594175s] snooze(0.123)-> None
[0.12556291s] snooze(0.123)-> None


In [13]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
for i in range(3):
    snooze(.123)

snooze: 0.13457107543945312s
snooze: 0.12633204460144043s
snooze: 0.12441277503967285s
